In [ ]:
!pip install selenium
!pip install tqdm
!pip install openpyxl

In [ ]:
import time # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm # for문 돌릴 때 진행상황을 %게이지로 알려준다.
from selenium import webdriver
import openpyxl

import urllib.request
import re

# 에러
from selenium.common.exceptions import NoSuchElementException # element 오류처리

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 액셀에서 검색할 데이터들 불러오기
load_datas = openpyxl.load_workbook("data1.xlsx")
datas = load_datas.active
column_A_datas = datas['A'][301:401] # 파일이름
column_B_datas = datas['B'][301:401] # 검색 키워드

crawling_names = []
for i in range(1, len(column_B_datas)):
    crawling_names.append((column_A_datas[i].value, column_B_datas[i].value))

In [ ]:
# v1
# ex) 신라면 블랙, 신라면을 고려안한 크롤링

# 네이버 api 사용할 때 필요한 값
client_id = "84DIX0pyMLLkcG2uuHia"
client_secret = "CRf5UovTxF"

for file_name, crawling_name in crawling_names:
    query_txt = '"' + crawling_name + '"' # 검색할 값을 따옴표 안에 넣어서 검색해주세요 (ex: "신라면 블랙") ( 따옴표 안에 넣고 검색시 이 값이 무조건 존재한 블로그들만 나타내는 걸로 알고 있어요!?)
    encText = urllib.parse.quote(query_txt) # 네이버 검색창 검색 키워드 

    blog_links = []

    for start in range(1, 200, 100): # 블로그 200개 기준으로 작성했습니다
        stringStart = str(start) # string형태여야 합니다
        url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&start="+stringStart+"&display=100" # 네이버 검색 api, start: 시작, display: 보여줄 갯수(100이 최대)

        # start(블로그 몇 번째인지?, max값 1000)와 display(한번 검색에 몇 개의 결과를 띄울건지, max값 100)조절해주기
        # start는 1, 101, 201 이렇게 시작해주세요
        # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            #print(response_body.decode('utf-8'))
        else:
            print("Error Code:" + rescode)

        body = response_body.decode('utf-8')
        body = body.replace('"','')

        #블로그 링크들 추출
        links = re.findall('link: (.*?),\ndescription',body)
        print('추출 링크 수: ',len(links),'개')

        for link in links:
            link_a = link.replace('\\','') # 링크 안의 '\\' 제거
            link_b = link_a.replace('?Redirect=Log&logNo=','/') # 링크안에 ?Redirect=Log&logNo= 제거
            blog_links.append(link_b) # 현재 링크를 배열에 추가 밑에 크롤링에서 사용

    print('총 링크 갯수:',len(blog_links),'개')

    #본문 크롤링
    path = "chromedriver.exe" # 크롬 버전에 맞는 크롬 드라이버를 실행하는 파일 위치랑 똑같은 곳에 놔둬주는게 편해요
    driver = webdriver.Chrome(path)

    # 블로그 링크를 하나씩 불러와서 크롤링
    contents = ''
    for i in tqdm(blog_links, desc=file_name): 
        driver.get(i) #불러온 링크에 접속
        driver.implicitly_wait(2) # 엔진 자체에서 로딩(파싱?)되는 시간을 기다려주는 것...?
        # time.sleep(1) # driver.implicitly_wait만 써도 괜찮은지 모르겠어서 time.sleep도 넣어놓은 거예요..

        print(i+' 링크하고 있습니다')
        try: # 네이버 블로그
            driver.switch_to.frame("mainFrame")  # 1)iframe접근 
            blog_contents = driver.find_element_by_css_selector('div.se-main-container').text
            contents += blog_contents + '\n==========================================================================================\n'
        except NoSuchElementException:
            blog_contents = driver.find_element_by_css_selector('div#content-area').text
            contents += blog_contents + '\n==========================================================================================\n'
        except NoSuchElementException: 
            blog_contents = driver.find_element_by_css_selector('div.contents_style').text # 티스토리인 걸로 알고 있어요 (제가 한건데 기억이 안나요 ㅈㅅㅈㅅ;;;)
            contents += blog_contents + '\n==========================================================================================\n'
        except: # 그 외
            pass

    # 링크 내용을 txt 파일로 저장 
    text_content = open(file_name+".txt", 'w', encoding='utf-8') # 파일이름 + 확장자명 : txt
    text_content.write(contents)  
    text_content.close()
    print(query_txt+" 저장 완료")